In [5]:
import pandas as pd 
from kiwipiepy import Kiwi
import numpy as np
import re
import pickle
from module.lastdance import Algorithm

ModuleNotFoundError: No module named 'module'

In [2]:
jh = JH()


In [ ]:
with open("nnp_list.pkl", "rb") as r :
    nnp_list = pickle.load(r)

with open("WD.StandardDict.p", "rb") as r :
    StandradDict = pickle.load(r)

with open(r"WD.WordToStandard.p", "rb") as r :
    WordToStandard = pickle.load(r)

In [1]:
class Algorithm:
    def __init__(self) -> None:
        self.stop_words = ["정식","안주"] 
        self.option_list = ["추가","선택", "많이", "적게", "조금","빼고", "리필", "기본"]


    # stop words algorithm / Step 1 #
    def is_stop_words(self, stop_word_text:str, stop_words_param:list) -> str:
        """_summary_

        Args:
            stop_word_text (str): _description_
            stop_words_param (list): _description_

        Returns:
            str: _description_
        """
        result = jh.kiwi.space(stop_word_text).split(" ")
        for i in result :
            if i in stop_words_param :
                pat = re.compile(f"{i}")
                text = re.sub(pat, "", stop_word_text)
        return text.strip()
    # apply 선언 함수
    stop_words_func = lambda self, txt : self.is_stop_words(txt, self.stop_words)
    
    # option check algorithm / Step 2 #
    def is_option(self, option_text:str, option_li:list) ->str:
        """_summary_

        Args:
            option_text (str): _description_
            option_li (list): _description_

        Returns:
            str: _description_
        """
        result = jh.kiwi.space(option_text).split(" ")
        for i in result :
            if i in option_li :
                pat = re.compile(f"{i}")
                valiable = "".join(pat.findall(option_text))
                return " ".join(result), f"옵션 {valiable}"
    # apply 선언 함수
    option_func = lambda self, txt : self.is_option(txt, self.option_list)

    # special_character algorithm / Step 3 #
    def special_char(self, special_text:str) -> str: 
        """_summary_

        Args:
            special_text (str): _description_

        Returns:
            str: _description_
        """
        split_result = re.split(r"\+|\&|\/|\,", special_text)
        if len(split_result) >= 2 :
            combine_text = [split_result[split_result.index(i)].strip() for i in split_result]
            pattern = re.compile(r"[^\w\s]+")
            return [pattern.sub("", i ) for i in combine_text]
        else : 
            pattern = re.compile(r"[^\w\s]+")
            return jh.kiwi.space(pattern.sub("", special_text))

    # unit algorithm / Step 4 #
    extract_morph = lambda self, x : jh.morph(x)

    # space algorithm / Step 5 #
    spacing_text = lambda self, x : jh.kiwi.space(x)

    # remove_nnp/ Step 6 #
    def nnp_extract(self, text:str, nnp_list:list) -> str :
        """_summary_

        Args:
            text (str): _description_
            nnp_list (list): _description_

        Returns:
            str: _description_
        """
        res = []
        for i in text.split() :      
            if i not in nnp_list :
                res.append(i)
        return " ".join(res)

    nnp_remove = lambda self, x: self.nnp_extract(x, nnp_list)


    # std words / Step 7 #
    def word_standard(self,txt:str) -> str :
        """_summary_

        Args:
            txt (str): _description_

        Returns:
            str: _description_
        """
        res = []
        for word in txt.split(' '):
            try:
                res.append(WordToStandard[word])
            except :
                res.append(word)
        return " ".join(res)
        
    # data standard / Step 8 #
    def member_menu(self, txt:str) -> str :
        """_summary_

        Args:
            txt (str): _description_

        Returns:
            str: _description_
        """
        score_ls = []
        word_ls = txt.split(' ')
        for word in word_ls:
            try:
                score_ls.append(StandradDict[word]['mean'])
            except:
                score_ls.append(10)
        return word_ls[np.array(score_ls).argmax()], word_ls[np.array(score_ls).argmin()]

In [2]:
algo = Algorithm()

In [3]:
algo.member_menu("초콜릿 라떼")

NameError: name 'np' is not defined

#### 불용어 알고리즘

In [42]:
stop_words = ["정식","안주"]

def is_stop_words(text, stop_words) :
    result = jh.kiwi.space(text).split(" ")
    for i in result :
        if i in stop_words :
            pat = re.compile(f"{i}")
            text = re.sub(pat, "", text)
    return text.strip()

func = lambda txt : is_stop_words(txt, stop_words)

#### 옵션 알고리즘

In [44]:
option_list = ["추가","선택", "많이", "적게", "조금","빼고", "리필"]
def is_option(text, option_li) :
    result = jh.kiwi.space(text).split(" ")
    for i in result :
        if i in option_li :
            pat = re.compile(f"{i}")
            valiable = "".join(pat.findall(text))
            return " ".join(result), f"옵션 {valiable}"

option_func = lambda txt : is_option(txt, option_list)

#### 특수문자 알고리즘

In [46]:
def special_char(text) : 
    split_result = re.split(r"\+|\&|\/|\,", text)
    if len(split_result) >= 2 :
        combine_text = [split_result[split_result.index(i)].strip() for i in split_result]
        pattern = re.compile(r"[^\w\s]+")
        return [pattern.sub("", i ) for i in combine_text]
        # text_token = [j for i in kiwi.tokenize(combine_text) for j in i]
        # cp = tuple(cp_data.form for cp_data in text_token if cp_data.tag.startswith("N"))
        # return cp
    else : 
        pattern = re.compile(r"[^\w\s]+")
        return jh.kiwi.space(pattern.sub("", text))


#### 단위 처리 알고리즘

In [52]:
jh.morph("과자")

('과자', '')

#### Spacing aloritm

In [53]:
jh.kiwi.space("안녕하세요지금이문장은스페이스알고리즘입니다")

'안녕하세요 지금 이 문장은 스페이스 알고리즘입니다'

#### 고유명사 알고리즘

In [47]:
def nnp_extract(text, nnp_list) :
    res = []
    # split_text = kiwi.space(text) 
    for i in text.split() :  
    # for i in split_text.split() :    
        if i not in nnp_list :
            res.append(i)
    return " ".join(res)
                     

#### 단어 표준화 알고리즘

In [51]:
def word_standard(txt) :
    res = []
    for word in txt.split(' '):
        try:
            res.append(WordToStandard[word])
        except :
            res.append(word)
    return " ".join(res)
        

#### 데이터 표준화 알고리즘

In [90]:
text = "아이스 카페라떼"

In [91]:
def member_menu(txt):
    score_ls = []
    word_ls = txt.split(' ')
    for word in word_ls:
        try:
            score_ls.append(StandradDict[word]['mean'])
        except:
            score_ls.append(10)
    return word_ls[np.array(score_ls).argmax()], word_ls[np.array(score_ls).argmin()]

In [8]:
pat = re.compile(r"\([^)]*\)|\[[^)]*\]")

In [9]:
pat.findall("[커피에반하다] 허니버터브레드 (기본)")

['[커피에반하다]', '(기본)']

In [ ]:
my_array = data.str.extractall(r"(\([^)]*\)|\[[^)]*\]|\([^)]\([^)]\))").values
my_list = my_array.tolist()